##### Copyright 2018 The TensorFlow Authors.


In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.



# Normalizations

<table class="tfo-notebook-buttons" align="left">

  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/addons/blob/master/examples/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/addons/blob/master/examples/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>



## Overview
This notebook gives a brief introduction into the [normalization layers](https://github.com/tensorflow/addons/blob/master/tensorflow_addons/layers/normalizations.py) of tensorflow addons. Currently supported layers are:
* **Group Normalization**
* **Layer Normalization**
* **Instance Normalization**

The basic idea behind these layers is to normalize the output of a activation layer to improve the convergence during training. In contrast to [batch normalization](https://keras.io/layers/normalization/) these normalizations do not work on batches, but they normalize the activations of a single sample, making them suitable for recurrent neual networks as well. 

Typically the normalization is performed by calculating the mean and the standard deviation of a subgroup of you input tensor. It is also possible to apply a scale and an offset factor to this as well.


$y_{i} = \frac{\gamma ( x_{i} - \mu )}{\sigma }+ \beta$

$ y$ : Output

$x$ : Input

$\gamma$ : Scale factor

$\mu$: mean

$\sigma$: standard deviation

$\beta$: Offset factor


The following image demonstrates the difference between these techniques. Each subplot shows a input tensor, with N as the batch axis, C as the channel axis, and (H, W)
as the spatial axes (Height and Width of a picture for example). The pixels in blue are normalized by the same mean and variance, computed by aggregating the values of these pixels.

![](https://github.com/shaohua0116/Group-Normalization-Tensorflow/raw/master/figure/gn.png)

Source: (https://arxiv.org/pdf/1803.08494.pdf)

The weights gamma and beta are trainable in all normalization layers to compensate for the possible lost of representational ability. You can activate these factors by setting the `center` or the `scale` flag to `True`. Of course you can use `initializers`, `constraints` and `regularizer` for `beta` and `gamma` to tune these values during the training process. 

##Setup

### Install Tensorflow 2.0 and Tensorflow-Addons

In [1]:
!pip install tensorflow==2.0.0-beta1 
!pip install tfa-nightly
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import tensorflow_addons as tfa

     |████████████████████████████████| 87.9MB 1.2MB/s 
     |████████████████████████████████| 3.1MB 34.4MB/s 
     |████████████████████████████████| 501kB 46.1MB/s 
  Found existing installation: tensorflow 1.14.0rc1
    Uninstalling tensorflow-1.14.0rc1:
      Successfully uninstalled tensorflow-1.14.0rc1
     |████████████████████████████████| 563kB 4.7MB/s 


###Preparing Dataset

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


## Group Normalization Tutorial 

### Introduction
Group Normalization(GN) divides the channels of your pictures into smaller sub groups and normalizes these values of these based on there mean and variance. Since GN works on a single example this technique is batchsize independent. 

GN experimentally scored closed to batch normalization in Imageclassification tasks. It can be beneficial to use GN instead of Batch Normalization in case your overall batch_size is low, which would lead to bad performance of batch normalization  

###Example
Splitting 10 channels after a Conv2D layer into 5 subgroups in a standard "channels last" setting:

In [3]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # Groupnorm Layer
  tfa.layers.normalizations.GroupNormalization(groups=5, axis=3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

W0626 21:25:19.246333 139850196416384 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10000 samples
10000/10000 [==============================] - 9s 856us/sample - loss: 0.4905 - accuracy: 0.8524


## Instance Normalization Tutorial
### Introduction
Instance Normalization is special case of group normalization where the group size is the same size as the channel size (or the axis size).

Experimental results show that instance normalization performs well on style transfer when replacing batch normalization. Recently, instance normalization has also been used as a replacement for batch normalization in GANs.

### Example
Applying InstanceNormalization after a Conv2D Layer and using a uniformed initialized scale and offset factor.

In [5]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tfa.layers.normalizations.InstanceNormalization(axis=3, 
                                                  center=True, 
                                                  scale=True,
                                                  beta_initializer="random_uniform",
                                                  gamma_initializer="random_uniform"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

Train on 10000 samples
10000/10000 [==============================] - 7s 658us/sample - loss: 0.5463 - accuracy: 0.8327


## Layer Normalization Tutorial
### Introduction
Layer Normalization is special case of group normalization where the group size is 1. The mean and standard deviation is calculated from all activations of a single sample.

Experimental results show that Layer normalization is well suited for Recurrent Neural Networks, since it works batchsize independt.

### Example

Applying Layernormalization after a Conv2D Layer and using a scale and offset factor. 

In [4]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tf.keras.layers.LayerNormalization(axis=1 , center=True , scale=True),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

Train on 10000 samples
10000/10000 [==============================] - 8s 769us/sample - loss: 0.4453 - accuracy: 0.8728


## Literature
[Layer norm](https://arxiv.org/pdf/1607.06450.pdf)

[Instance norm](https://arxiv.org/pdf/1607.08022.pdf)

[Group Norm](https://arxiv.org/pdf/1803.08494.pdf)

[Complete Normalizations Overview](http://mlexplained.com/2018/11/30/an-overview-of-normalization-methods-in-deep-learning/)